In [1]:
import sqlite3
import csv
from Hopper import *

In [2]:
verbose = True
h = dbms(verbose=verbose)

DROP TABLE IF EXISTS i_ids;

CREATE TABLE i_ids (
	i_id TEXT NOT NULL,
	PRIMARY KEY (i_id)
);

DROP TABLE IF EXISTS i_windows;

CREATE TABLE i_windows (
	i_id TEXT NOT NULL,
	i_st INTEGER NOT NULL,
	i_et INTEGER NOT NULL,
	PRIMARY KEY (i_id),
	FOREIGN KEY (i_id) REFERENCES i_ids (i_id)
);

DROP TABLE IF EXISTS i_partitions;

CREATE TABLE i_partitions (
	i_id TEXT NOT NULL,
	partition TEXT NOT NULL,
	PRIMARY KEY (i_id),
	FOREIGN KEY (i_id) REFERENCES i_ids (i_id)
);

DROP TABLE IF EXISTS i_relcal;

CREATE TABLE i_relcal (
	i_id TEXT NOT NULL,
	i_st INTEGER NOT NULL,
	i_et INTEGER NOT NULL
);



In [3]:
tc = table_config("characteristics_0", 
                  ["age", "sex", "eth", "bin_ldc", "job"], 
                  ["real", "bin", "ldc", "ldc", "hdc"],
                  has_times=False,
                  primary_key=True)
h.create_fvm_with_csv(tc, "data/characteristics_0.csv", delimiter=',')


_tc = table_config("samples_0", 
                  ["SBP", "DBP", "type"], 
                  ["real", "real", "ldc"], 
                  has_times=True,
                  primary_key=False)
h.create_fvm_with_csv(_tc, "data/samples_0.csv", delimiter=',')

_tc = table_config("samples_1", 
                  ["tmps", "hrs", "sbps", "rrs", "dbps", "sats", "wgts"],
                  ["real", "real", 'real', "real", "real", "real", "real"], 
                  has_times=True,
                  primary_key=False)
h.create_fvm_with_csv(_tc, "data/samples_1.csv", delimiter=',')


_tc = table_config("samples_2", 
                  ["dx"],
                  ["hdc"], 
                  has_times=True,
                  primary_key=False)
h.create_fvm_with_csv(_tc, "data/samples_2.csv", delimiter=',')


_tc = table_config("samples_3", 
                  ["outcome"],
                  ["bin"], 
                  has_times=True,
                  primary_key=False)
h.create_fvm_with_csv(_tc, "data/samples_3.csv", delimiter=',')

DROP TABLE IF EXISTS characteristics_0;

CREATE TABLE characteristics_0 (
	i_id TEXT NOT NULL,
	age REAL NOT NULL,
	sex TEXT NOT NULL,
	eth TEXT NOT NULL,
	bin_ldc TEXT NOT NULL,
	job TEXT NOT NULL,
	PRIMARY KEY (i_id),
	FOREIGN KEY (i_id) REFERENCES i_ids (i_id)
);

INSERT OR IGNORE INTO i_ids (i_id) VALUES (?);

INSERT  INTO characteristics_0 (i_id, age, sex, eth, bin_ldc, job) VALUES (?, ?, ?, ?, ?, ?);

CREATE INDEX i_index_characteristics_0_i_id ON characteristics_0(i_id);

CREATE INDEX i_index_characteristics_0_age ON characteristics_0(age);

CREATE INDEX i_index_characteristics_0_sex ON characteristics_0(sex);

CREATE INDEX i_index_characteristics_0_eth ON characteristics_0(eth);

CREATE INDEX i_index_characteristics_0_bin_ldc ON characteristics_0(bin_ldc);

CREATE INDEX i_index_characteristics_0_job ON characteristics_0(job);

10000 rows loaded

DROP TABLE IF EXISTS samples_0;

CREATE TABLE samples_0 (
	i_id TEXT NOT NULL,
	i_st INTEGER NOT NULL,
	i_et INTEGER NOT NULL,
	SBP RE

In [4]:
#h.dew_it()

In [5]:
h.set_windows([['00000', 2, 5], ['00001', 3, 7]],)

INSERT  INTO i_windows (i_id, i_st, i_et) VALUES (?, ?, ?);

gen_range_table_sql called: af=None, bf=None
INSERT OR IGNORE INTO i_windows 
SELECT * FROM (
	SELECT i_id, MIN(i_st) AS i_st, MAX(i_et) AS i_et
	FROM (
	    SELECT i_id, i_st, i_et
		FROM samples_0
		UNION
			SELECT i_id, i_st, i_et
		FROM samples_1
		UNION
			SELECT i_id, i_st, i_et
		FROM samples_2
		UNION
			SELECT i_id, i_st, i_et
		FROM samples_3
		
	)
	GROUP BY i_id
);



In [6]:
h.set_partitions([['00000', 'dev'], ['00001', 'dev']])

INSERT  INTO i_partitions (i_id, partition) VALUES (?, ?);

INSERT OR IGNORE INTO i_partitions 
SELECT * FROM (
	SELECT i_id, CASE 
		 WHEN r <= 0.8 THEN "train" 
		 WHEN r <= 0.9 THEN "dev" 
		 WHEN r <= 1.0 THEN "test" 
		 ELSE "test" 
	END
	FROM (SELECT i_id, ABS(RANDOM())/(9223372036854775807.0) AS r FROM i_ids)
);



In [7]:
h.create_windows_views()

CREATE VIEW win_samples_0 AS
    SELECT samples_0.i_id,
	MAX(samples_0.i_st, i_windows.i_st) AS i_st,
	MIN(samples_0.i_et, i_windows.i_et) AS i_et,
	SBP,
	DBP,
	type
	FROM (
	    samples_0 JOIN i_windows ON samples_0.i_id=i_windows.i_id
	)
	WHERE NOT (i_windows.i_et < samples_0.i_st OR
	           i_windows.i_st >= samples_0.i_et)
	
;

CREATE VIEW win_samples_1 AS
    SELECT samples_1.i_id,
	MAX(samples_1.i_st, i_windows.i_st) AS i_st,
	MIN(samples_1.i_et, i_windows.i_et) AS i_et,
	tmps,
	hrs,
	sbps,
	rrs,
	dbps,
	sats,
	wgts
	FROM (
	    samples_1 JOIN i_windows ON samples_1.i_id=i_windows.i_id
	)
	WHERE NOT (i_windows.i_et < samples_1.i_st OR
	           i_windows.i_st >= samples_1.i_et)
	
;

CREATE VIEW win_samples_2 AS
    SELECT samples_2.i_id,
	MAX(samples_2.i_st, i_windows.i_st) AS i_st,
	MIN(samples_2.i_et, i_windows.i_et) AS i_et,
	dx
	FROM (
	    samples_2 JOIN i_windows ON samples_2.i_id=i_windows.i_id
	)
	WHERE NOT (i_windows.i_et < samples_2.i_st OR
	           i_windows.i

In [8]:
h.create_partition_views_prior_to_filter()

SELECT DISTINCT(partition) FROM i_partitions;

partitions: dev,train,test

CREATE VIEW i_partitions_dev_org_characteristics_0 AS
    SELECT * FROM characteristics_0
	WHERE i_id IN (
	    SELECT i_id
	    FROM i_partitions
	    WHERE partition='dev'
	)
	
;

CREATE VIEW i_partitions_train_org_characteristics_0 AS
    SELECT * FROM characteristics_0
	WHERE i_id IN (
	    SELECT i_id
	    FROM i_partitions
	    WHERE partition='train'
	)
	
;

CREATE VIEW i_partitions_test_org_characteristics_0 AS
    SELECT * FROM characteristics_0
	WHERE i_id IN (
	    SELECT i_id
	    FROM i_partitions
	    WHERE partition='test'
	)
	
;

CREATE VIEW i_partitions_dev_win_samples_0 AS
    SELECT * FROM win_samples_0
	WHERE i_id IN (
	    SELECT i_id
	    FROM i_partitions
	    WHERE partition='dev'
	)
	
;

CREATE VIEW i_partitions_train_win_samples_0 AS
    SELECT * FROM win_samples_0
	WHERE i_id IN (
	    SELECT i_id
	    FROM i_partitions
	    WHERE partition='train'
	)
	
;

CREATE VIEW i_partitions_test

In [9]:
h.fit_filter(partition='train', percentile_cutoff=0.05)

SELECT COUNT(*) FROM i_partitions_train_org_characteristics_0;

SELECT AVG(age) AS avg,
       SUM((age-(SELECT AVG(age) FROM i_partitions_train_org_characteristics_0))*
       (age-(SELECT AVG(age) FROM i_partitions_train_org_characteristics_0)))
       /(COUNT(age)-1) AS var
FROM i_partitions_train_org_characteristics_0
;

SELECT sex, COUNT(*) AS n
FROM i_partitions_train_org_characteristics_0
GROUP BY sex
ORDER BY n DESC
;


SELECT eth, COUNT(*) AS n
FROM i_partitions_train_org_characteristics_0
GROUP BY eth
ORDER BY n DESC
;


SELECT bin_ldc, COUNT(*) AS n
FROM i_partitions_train_org_characteristics_0
GROUP BY bin_ldc
ORDER BY n DESC
;


SELECT job, COUNT(*) AS n
FROM i_partitions_train_org_characteristics_0
GROUP BY job
ORDER BY n DESC
;


SELECT COUNT(*) FROM i_partitions_train_win_samples_0;

SELECT AVG(SBP) AS avg,
       SUM((SBP-(SELECT AVG(SBP) FROM i_partitions_train_win_samples_0))*
       (SBP-(SELECT AVG(SBP) FROM i_partitions_train_win_samples_0)))
       /(COUNT(SBP)-1

In [10]:
h.set_relcal()

SELECT min(i_st) AS i_st, max(i_et) AS i_et
FROM (
    SELECT i_id, i_st, i_et
	FROM samples_0
	UNION
		SELECT i_id, i_st, i_et
	FROM samples_1
	UNION
		SELECT i_id, i_st, i_et
	FROM samples_2
	UNION
		SELECT i_id, i_st, i_et
	FROM samples_3
	
);

INSERT  INTO i_relcal (i_id, i_st, i_et) VALUES (?, ?, ?);



In [11]:
h.fil_agg()

CREATE VIEW fil_characteristics_0 AS
    SELECT i_id,
	CASE
	    WHEN 25.5177928478575>age THEN 25.0
	    WHEN age>64.58496404437307 THEN 65.0 ELSE age
	END AS age,
	CASE
	    WHEN sex IN ('M') THEN 1
	    ELSE 0
	END AS sex,
	CASE
	    WHEN eth IN ('e0', 'e1', 'e2', 'e3', 'e4', 'e5') THEN eth
	    ELSE '_OTHER_'
	END AS eth,
	CASE
	    WHEN bin_ldc IN ('A', 'B') THEN bin_ldc
	    ELSE '_OTHER_'
	END AS bin_ldc,
	CASE
	    WHEN job IN ('j000', 'j001', 'j002', 'j003', 'j004', 'j005', 'j006', 'j009', 'j007', 'j008', 'j012', 'j010', 'j011', 'j013', 'j015', 'j017', 'j014', 'j016', 'j020', 'j022', 'j018', 'j021', 'j023', 'j027', 'j019', 'j026', 'j032', 'j025', 'j033', 'j029', 'j039', 'j045', 'j031', 'j035', 'j037', 'j028', 'j043', 'j024', 'j036', 'j041', 'j038', 'j030', 'j034', 'j066', 'j042', 'j055', 'j051', 'j048', 'j062', 'j044', 'j098', 'j086', 'j079', 'j052', 'j047', 'j046', 'j064', 'j058', 'j040', 'j069', 'j053', 'j094', 'j075', 'j072', 'j060', 'j057', 'j054', 'j101', 'j076', 'j073', 

In [12]:
h.create_partition_views_prior_to_normalization()

SELECT DISTINCT(partition) FROM i_partitions;

partitions: dev,train,test

CREATE VIEW i_partitions_dev_ohe_characteristics_0 AS
    SELECT * FROM ohe_characteristics_0
	WHERE i_id IN (
	    SELECT i_id
	    FROM i_partitions
	    WHERE partition='dev'
	)
	
;

CREATE VIEW i_partitions_train_ohe_characteristics_0 AS
    SELECT * FROM ohe_characteristics_0
	WHERE i_id IN (
	    SELECT i_id
	    FROM i_partitions
	    WHERE partition='train'
	)
	
;

CREATE VIEW i_partitions_test_ohe_characteristics_0 AS
    SELECT * FROM ohe_characteristics_0
	WHERE i_id IN (
	    SELECT i_id
	    FROM i_partitions
	    WHERE partition='test'
	)
	
;

CREATE VIEW i_partitions_dev_agg_samples_0 AS
    SELECT * FROM agg_samples_0
	WHERE i_id IN (
	    SELECT i_id
	    FROM i_partitions
	    WHERE partition='dev'
	)
	
;

CREATE VIEW i_partitions_train_agg_samples_0 AS
    SELECT * FROM agg_samples_0
	WHERE i_id IN (
	    SELECT i_id
	    FROM i_partitions
	    WHERE partition='train'
	)
	
;

CREATE VIEW i_par

In [13]:
%time h.fit_normalization(partition='train')

SELECT AVG(age),
COUNT(age) 
FROM i_partitions_train_ohe_characteristics_0;

SELECT SUM((age-(45.06754385964912))*(age-(45.06754385964912)))/(7980-1) 
FROM i_partitions_train_ohe_characteristics_0;

SELECT AVG(avg_SBP),
COUNT(avg_SBP),
AVG(min_SBP),
COUNT(min_SBP),
AVG(max_SBP),
COUNT(max_SBP),
AVG(avg_DBP),
COUNT(avg_DBP),
AVG(min_DBP),
COUNT(min_DBP),
AVG(max_DBP),
COUNT(max_DBP),
AVG(count),
COUNT(count) 
FROM i_partitions_train_agg_samples_0;

SELECT SUM((avg_SBP-(120.02261904244791))*(avg_SBP-(120.02261904244791)))/(103487-1),
SUM((min_SBP-(118.74816564069697))*(min_SBP-(118.74816564069697)))/(103487-1),
SUM((max_SBP-(121.29622532078133))*(max_SBP-(121.29622532078133)))/(103487-1),
SUM((avg_DBP-(80.00235053576667))*(avg_DBP-(80.00235053576667)))/(103487-1),
SUM((min_DBP-(78.73336831408065))*(min_DBP-(78.73336831408065)))/(103487-1),
SUM((max_DBP-(81.27189394965903))*(max_DBP-(81.27189394965903)))/(103487-1),
SUM((count-(1.5422226946379738))*(count-(1.5422226946379738)))/(103487-1)

In [14]:
%time h.fit_normalization(partition='train', via_sql_qds=False)

SELECT age
FROM i_partitions_train_ohe_characteristics_0
;

SELECT avg_SBP, min_SBP, max_SBP, avg_DBP, min_DBP, max_DBP, count
FROM i_partitions_train_agg_samples_0
;

SELECT avg_tmps, min_tmps, max_tmps, avg_hrs, min_hrs, max_hrs, avg_sbps, min_sbps, max_sbps, avg_rrs, min_rrs, max_rrs, avg_dbps, min_dbps, max_dbps, avg_sats, min_sats, max_sats, avg_wgts, min_wgts, max_wgts, count
FROM i_partitions_train_agg_samples_1
;

SELECT count
FROM i_partitions_train_agg_samples_2
;

SELECT count
FROM i_partitions_train_agg_samples_3
;

CPU times: user 11.8 s, sys: 102 ms, total: 11.9 s
Wall time: 12 s


In [15]:
h.normalize()

CREATE VIEW nrm_characteristics_0 AS
    SELECT i_id,
	(age-46.0)/0.0 as age,
	sex,
	eth_e0,
	eth_e1,
	eth_e2,
	eth_e3,
	eth_e4,
	eth_e5,
	eth__OTHER_,
	bin_ldc_A,
	bin_ldc_B,
	bin_ldc__OTHER_,
	job
	FROM ohe_characteristics_0
	
;

CREATE VIEW nrm_samples_0 AS
    SELECT i_id,
	i_st,
	i_et,
	(avg_SBP-80.36326989794998)/39.073828096734815 as avg_SBP,
	(min_SBP-85.38540290606761)/38.96663566136587 as min_SBP,
	(max_SBP-86.57065049041161)/37.49497050877817 as max_SBP,
	(avg_DBP-87.61502284977423)/39.288222925295074 as avg_DBP,
	(min_DBP-84.50788380986674)/40.6961527645471 as min_DBP,
	(max_DBP-86.75697880205409)/38.53049598124075 as max_DBP,
	avg_type_c0,
	avg_type_c1,
	avg_type_c2,
	avg_type_c3,
	avg_type_c4,
	avg_type__OTHER_,
	(count-88.33059642819819)/39.095763700612466 as count
	FROM agg_samples_0
	
;

CREATE VIEW nrm_samples_1 AS
    SELECT i_id,
	i_st,
	i_et,
	(avg_tmps-79.84123834696194)/45.46869372541694 as avg_tmps,
	(min_tmps-81.94450523104504)/47.248942842210994 as min_tmps,
	

In [16]:
h.execute("PRAGMA table_info(fil_characteristics_0);", size=10)

PRAGMA table_info(fil_characteristics_0);
(0, 'i_id', 'TEXT', 0, None, 0)
(1, 'age', '', 0, None, 0)
(2, 'sex', '', 0, None, 0)
(3, 'eth', '', 0, None, 0)
(4, 'bin_ldc', '', 0, None, 0)
(5, 'job', '', 0, None, 0)


In [17]:
#SELECT * FROM nrm_samples_3 WHERE i_id=='00000';
#SELECT * FROM agg_samples_3 WHERE i_id=='00000';
#SELECT * FROM ohe_samples_3 WHERE i_id=='00000';

In [18]:
#SELECT * FROM agg_samples_0 LIMIT 10;
#CREATE INDEX samples_0_index_type ON samples_0(type);
#SELECT * FROM i_partitions_dev_ohe_characteristics_0;
#["SBP", "DBP", "type"]

In [19]:
#h.interactive_session()